# Kickstarter one-hot Encoding to npz

### Extract the data from the csv

In [7]:
import numpy as np
from sklearn import preprocessing
from pickle import dump


In [ ]:
raw_csv_data = np.loadtxt('ks-projects-201801-one-hot.csv', delimiter = ',')
unscaled_inputs_all = raw_csv_data[:,1:]
targets_all = raw_csv_data[:,0]

### Balance the dataset

In [2]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

### Standardize the inputs

In [3]:
scaler = preprocessing.StandardScaler().fit(unscaled_inputs_equal_priors)
scaler

StandardScaler()

In [4]:
scaler.mean_

array([1.42078440e+02, 1.21598771e+04, 3.67868902e+04, 3.21497626e+01,
       4.50890888e+00, 9.10173438e-02, 1.22287663e-01, 7.86694993e-01,
       7.75596147e-02, 3.21549000e-02, 2.20842847e-02, 1.17188091e-02,
       8.02057974e-02, 5.67454977e-02, 1.68236735e-01, 6.20756657e-02,
       9.47749233e-02, 1.14012671e-02, 1.41219210e-01, 2.76979723e-02,
       1.02959188e-01, 7.79565421e-02, 3.32095928e-02])

In [5]:
scaler.scale_

array([1.07909827e+03, 1.08039035e+05, 1.01169771e+06, 1.14186753e+01,
       3.19138422e+02, 2.87633772e-01, 3.27617750e-01, 4.09641283e-01,
       2.67477328e-01, 1.76411344e-01, 1.46957712e-01, 1.07617278e-01,
       2.71611538e-01, 2.31355670e-01, 3.74076377e-01, 2.41292929e-01,
       2.92903802e-01, 1.06166276e-01, 3.48247533e-01, 1.64106047e-01,
       3.03905567e-01, 2.68103188e-01, 1.79183469e-01])

In [6]:
scaler.transform(unscaled_inputs_equal_priors)

array([[-0.13166404, -0.11255077, -0.03484533, ...,  2.95170904,
        -0.29077066, -0.18533849],
       [-0.11776355, -0.09014221, -0.00670842, ..., -0.33878678,
        -0.29077066, -0.18533849],
       [-0.12888394, -0.11051447,  0.00811815, ..., -0.33878678,
        -0.29077066, -0.18533849],
       ...,
       [-0.10293635, -0.09651935, -0.03542253, ...,  2.95170904,
        -0.29077066, -0.18533849],
       [-0.05938147, -0.06642217, -0.03188411, ..., -0.33878678,
        -0.29077066, -0.18533849],
       [-0.09830285, -0.08593234, -0.03371729, ..., -0.33878678,
        -0.29077066, -0.18533849]])

In [8]:
# Save the scaler
dump(scaler, open('scaler_kicsktarter_v1.pkl', 'wb'))

In [3]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

C:\Users\David\Anaconda3\envs\tf\lib\site-packages\sklearn\preprocessing\_data.py:174: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [4]:
scaled_inputs

array([[-0.13166404, -0.11255077, -0.03484533, ...,  2.95170904,
        -0.29077066, -0.18533849],
       [-0.11776355, -0.09014221, -0.00670842, ..., -0.33878678,
        -0.29077066, -0.18533849],
       [-0.12888394, -0.11051447,  0.00811815, ..., -0.33878678,
        -0.29077066, -0.18533849],
       ...,
       [-0.10293635, -0.09651935, -0.03542253, ...,  2.95170904,
        -0.29077066, -0.18533849],
       [-0.05938147, -0.06642217, -0.03188411, ..., -0.33878678,
        -0.29077066, -0.18533849],
       [-0.09830285, -0.08593234, -0.03371729, ..., -0.33878678,
        -0.29077066, -0.18533849]])

### Shuffle the data

In [5]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Split the dataset into train, validation, and test

In [6]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

105720.0 211625 0.49956290608387477
13290.0 26453 0.5024004838770649
13256.0 26454 0.5010962425342104


### Save the three datasets in *.npz

In [7]:
np.savez('kickstarter_data_train', inputs=train_inputs, targets=train_targets)
np.savez('kickstarter_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('kickstarter_data_test', inputs=test_inputs, targets=test_targets)